In [56]:
import pandas as pd 
import numpy as np
import tweepy as tw
import spacy
import re
import psycopg2

In [57]:
api_key = 'xxx'
api_secret_key = 'xxx'
bearer_token =  'xxx'

In [58]:
access_token = 'xxx'
access_token_secret = 'xxx'

In [59]:
auth = tw.OAuthHandler(api_key, api_secret_key)
#auth = tw.AppAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [60]:
#Connect to an existing database on RDS
#conn = psycopg2.connect("dbname=test user=postgres")
connection = psycopg2.connect(user = "postgres",
                              password = "xxx",
                              host = "xxx",
                              port = "5432",
                              database = "dbname")
cursor = connection.cursor()
print(connection.get_dsn_parameters(),"\n")

{'user': 'postgres', 'dbname': 'dbname', 'host': 'database-1.cc58nuqy1vfs.eu-central-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [61]:
nlp = spacy.load("pl_core_news_sm")
all_stopwords = nlp.Defaults.stop_words

### Function to clean a tweet

In [62]:
def clean_tweet(tweet):
    tweet = " ".join(re.sub(r'https?:\/\/.*[\r\n]*',"",tweet).split()) # remove urls
    tweet = [word for word in tweet.lower().split() if not word in all_stopwords] # remove stopwords
    regex = re.compile(r'.*@')
    tweet = [i for i in tweet if not regex.match(i)] # remove hashtags
    regex = re.compile(r'^\D+$')
    tweet = [word for word in tweet if regex.match(word)]
    tweet = [token.lemma_ for token in nlp(" ".join(tweet)) if not token.is_punct]
    tweet = ' '.join(tweet)
    return tweet

In [63]:
clean_tweet('Cześć! Co dzisiaj robią inne partie? Czy coś inwestujecie? #pudło')

'cześć robić partia inwestować pudło'

### Count points

In [64]:
# vocabulary
economy = ['gospodarka', 'inwestycja', 'pieniądz', 'mieszkanie', 'zawodowy', 'rolnictwo', 'praca', 'bezrobocie', 'przemysł', 'emerytura', 'płaca', 'innowacja', 'ulga', 'podatek', 'podatkowy', 'infrastruktura', 'inflacja', 'kryzys', 'lokator', 'lokatorski', 'zawodowy']
health = ['szpital', 'pandemia', 'zdrowie', 'szczepienie', 'maseczka', 'pacjent', 'epidemia', 'epidemiczny', 'lekarz', 'proces', 'medyczny']
justice = ['sąd', 'trybunał', 'sprawiedliwość', 'konstytucja', 'wyrok', 'tk', 'ratyfikacja', 'referendum', 'sanitarny']
social = ['lgbtq', 'kościół', 'aborcja', 'religia', 'lgbt', 'gej', 'kobieta', 'stambulska', 'homoseksualista']

vocab = dict({'economy': economy, 
              'health': health,
             'justice': justice,
             'social': social})

In [65]:
def count_points(tweet):
    """A function which awards a point to a tweet if any word from a category's dictionary is in it"""
    return [int(any(word in tweet for word in dictionary)) for dictionary in list(vocab.values())]

In [73]:
points = count_points('gospodarxka jakoś się kręci, ale pandemia nie ustępuje'); points


[0, 1, 0, 0]

In [75]:
parties = ['__Lewica', 'pisorgpl', 'Platforma_org', 'KONFEDERACJA_', 'nowePSL', 'PL_2050']
parties_id = ['66324459', '15877628', '53003895', '1093510047605374976', '2426963910', '1295647949238697987']
dict(zip(parties, parties_id))

{'__Lewica': '66324459',
 'pisorgpl': '15877628',
 'Platforma_org': '53003895',
 'KONFEDERACJA_': '1093510047605374976',
 'nowePSL': '2426963910',
 'PL_2050': '1295647949238697987'}

In [68]:
# class StreamListener(tw.StreamListener):
#     def on_status(self, status):
#         text_clear = clean_tweet(status.text)
#         #print(text_clear)
#         return [status.user.screen_name, status.user.name, 
#                 status.id_str, status.created_at, 
#                 status.favorite_count, status.retweet_count,
#                status.text, text_clear]
#     def on_error(self, status_code):
#         if status_code == 420:
#             return False

In [69]:
# class StreamListener(tw.StreamListener):
#     def on_status(self, status):
#         text_clear = clean_tweet(status.text)
        
#         #print(text_clear)
#         try:
#             cursor.execute("""INSERT INTO test2 (screen_name, name, id, created_at, favorite_count, retweet_count, text, text_clear) 
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s);""", 
#                            (status.user.screen_name, status.user.name, 
#                     int(status.id_str), status.created_at, 
#                     status.favorite_count, status.retweet_count,
#                    status.text, text_clear))
#             connection.commit()
# #             print(status.user.screen_name, status.user.name, 
# #                     int(status.id_str), status.created_at, 
# #                     status.favorite_count, status.retweet_count,
# #                    status.text, text_clear)
#             #print('twitt added')
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             connection.rollback()
#             #cursor.close()
#             return 1

#     def on_error(self, status_code):
#         if status_code == 420:
#             return False

In [86]:
class StreamListener(tw.StreamListener):
    def on_status(self, status):
        if status.user.screen_name not in parties:
            return
        text_clear = clean_tweet(status.text)
        points = count_points(text_clear)
        print(text_clear)
        try:
            cursor.execute("""INSERT INTO test2 (screen_name, name, id, created_at, 
            text, text_clear,
            score_economy, score_health, score_justice, score_social) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", 
                           (status.user.screen_name, status.user.name, 
                    int(status.id_str), status.created_at, 
                   status.text, text_clear,
                           points[0], points[1], points[2], points[3]))
            connection.commit()
#             print(status.user.screen_name, status.user.name, 
#                     int(status.id_str), status.created_at, 
#                    status.text, text_clear)
            #print('twitt added')
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            connection.rollback()
            #cursor.close()
            return 1

    def on_error(self, status_code):
        if status_code == 420:
            return False

In [88]:
stream_listener = StreamListener()
stream = tw.Stream(auth=api.auth, listener=stream_listener)
#stream.filter(track=["kaczyński"])
stream.filter(follow=parties_id)

polski okazuje przodownik praca temacie paszport covidowych jesteśmy nadgorliwiec kt
rt to dziać sprawa kopalnia tur obrazować soczewka patologie władza skłócenie polski sąsiad niekom
rt wniosek klub koalicja obywatelski odbywać posiedzenie komisja ds kontrola państwowy udziać
apartheid szczepionkowy nazwać kazik miejsce polsce policja urząd wojsko
🎙 📣 zmiana polskiład skorzysta mln polak polaków reforma korzystny ne
rt wspólnie spotkać branża hotelowy koncertowy fitness ciężko dotkniętymi ch
wytężony wielomiesięczny propaganda szczepionkowy rząd zaczynać odciskać piętno relacjac
rt 🎥 sejm rp odbywać kolejny spotkanie cykl polka poświęcony sprawa kobieta pn kobieta samorządzie tran
gdyby możliwe wniosek odwołanie marszałek terleckiego złożyć piątka regulamin sejm mówić jeden
po wpis pojawić wpis przedstawiciel ko składać wniosek zapowiedzieć sobota
lewica zebrać podpis sporządzić wniosek wspólnie poseł dziennikarz dowiedzieć się ż
tydzień przedstawiciel klub stanąć wspólnie obrona bran

ProtocolError: ("Connection broken: ConnectionResetError(10054, 'Istniejące połączenie zostało gwałtownie zamknięte przez zdalnego hosta', None, 10054, None)", ConnectionResetError(10054, 'Istniejące połączenie zostało gwałtownie zamknięte przez zdalnego hosta', None, 10054, None))